# $N$- body Simulation

## Oshri Fatkiev

Simple $N$-body simulation using only gravity. To read about this type of simulations consider visiting https://en.wikipedia.org/wiki/N-body_simulation. 

To model how a collection of $N$ particles will interact in the presence of Newton's law of gravity, we can simply compute the acceleration acting on each particles from the force it feels from the remaining $N-1$ particles, update its positions then its velocity and keep iterating until the simulation ends.

The notebook organizes as follows:
- Deriving the acceleration $a_{ij}$ of each particle $i$ from particle $j$ due to Newton's gravity
- Implemeting a function calculating the acceleration of all the $N$ particles
- Deriving the terms for kinetic and potential energies $E_{k},E_{p}$ respectively of each particle
- Implementing a function calculating the $E_{k}$ and $E_{p}$
- Putting it all together and writing the simulation 

## Deriving the acceleration

As we know, the Newtonian force of gravity in vector form is given by:

$$ \boldsymbol{F_{21}} = -\frac{Gm_1m_2}{\lvert \boldsymbol{r_{21}} \rvert^{3}} \boldsymbol{r_{21}} $$

where ... Writing it in components, defining $\boldsymbol{r_{i}} = \left(x_i, y_i, z_i\right)$ for $i\in\{1,2\}$ we get:

$$ \boldsymbol{F_{12}}=-\frac{Gm_{1}m_{2}}{\left[\left(x_{2}-x_{1}\right)^{2}+\left(y_{2}-y_{1}\right)^{2}+\left(z_{2}-z_{1}\right)^{2}\right]^{3/2}}\left(x_{2}-x_{1},y_{2}-y_{1},z_{2}-z_{1}\right) $$

To make things look nicer we can define $dx=x_i-x_j$ thus the equation above can be written as

$$ \boldsymbol{F_{12}}=-\frac{Gm_{1}m_{2}}{\left(dx^{2}+dy^{2}+dz^{2}\right)^{3/2}}\left(dx,dy,dz\right) $$

Now, as we know from Newton's 2nd law that $\boldsymbol{F}=m\boldsymbol{a}$ we can multiply both sides by mass and we get the acceleration is simply

$$ a_{i}=-\frac{Gm_{j}}{\left(dx^{2}+dy^{2}+dz^{2}\right)^{3/2}}dx_{i} $$ 

where $m_{j}$ is the mass of particle $j$ exerts force on the particle we currently looking at. 

In simulations, its common to add a small term to the denomiantor, called *softening* which we will denote as $\epsilon$. The idea behind it is to make sure we avoid diverging when two particles getting close to each other. To read more about it, please refer to the wikipedia link above. If so, we can compute the accleration by simply using the equation:

$$ a_{i}=-\frac{Gm_{j}}{\left(dx^{2}+dy^{2}+dz^{2}+\epsilon^2\right)^{3/2}}dx_{i} $$

## Implemeting a function calculating the acceleration `get_acceleration`

First, let's import the packages we are going to use in this notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display, clear_output

Now we are ready to implement the `get_acceleration` function. The function **inputs**:
- `positions` - the 3d position components of all the particles in the simulation
- `mass` - the particles masses 
- `G` - the Newton's Gravitational constant (default value set to be 1) 
- `softening` - the softening length $\epsilon$ (default value is 0.1) 

**output**:
- `a` - the acceleration $x,y,z$ components of each particle.

In [ ]:
def get_acceleration(positions, mass, G=1, softening=0.1):
    # positions r = [x,y,z] for all particles
    x = positions[:, 0:1]
    y = positions[:, 1:2]
    z = positions[:, 2:3]

    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z

    # matrix that stores 1/r^3 for all particle pairwise particle separations 
    inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)
    inv_r3[inv_r3>0] = inv_r3[inv_r3>0]**(-3/2)

    ax = G * (dx * inv_r3) @ mass
    ay = G * (dy * inv_r3) @ mass
    az = G * (dz * inv_r3) @ mass
    
    # pack together the acceleration components
    a = np.hstack((ax,ay,az))

    return a

## Deriving the energies

Next, we would like to compute the kinetic and potential energies of each particle. The kinetic energy, $E_k$, is simply given by 

$$ E_k= \frac{1}{2}mv^2 $$

thus given the mass and velocity of each particle, we can compute it's kinetic energy. 
Next, the potential energy, $E_p$, from Newton's law of gravity is 

$$ E_p = -\frac{Gm_1m_2}{\left|\boldsymbol{r_{12}}\right|} $$


##  Implemeting a function calculating the energies `get_energy`

Moving on, let's implement the `get_energy` function. The function **inputs**:
- `positions` - the 3d position components of all the particles in the simulation
- `velocities` - the 3d velocity components of all the particles in the simulation
- `mass` - the particles masses 
- `G` - the Newton's Gravitational constant (default value set to be 1) 

**outputs**:
- `kinetic_energy` - the kinetic energy of the system
- `potential_energy` - the potential energy of the system 

In [ ]:
def get_energy(positions, velocities, mass, G=1):
    kinetic_energy = 0.5 * np.sum(np.sum(mass * velocities**2))

    # position r = [x,y,z] for all particles
    x = positions[:, 0:1]
    y = positions[:, 1:2]
    z = positions[:, 2:3]

    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z

    # matrix that stores 1/r for all particle pairwise particle separations 
    inv_r = np.sqrt(dx**2 + dy**2 + dz**2)
    inv_r[inv_r > 0] = 1.0 / inv_r[inv_r > 0]

    # sum over upper triangle, to count each interaction only once
    potential_energy = G * np.sum(np.sum(np.triu(-(mass * mass.T) * inv_r, 1)))
    
    return kinetic_energy, potential_energy

## Writing the simulation

Now, after writing all the necessary functions needed for the simulation, we can write the the simulation's parameters and the simulation's main loop

### The simulation's parameters

In [ ]:
seed = 17                # set the random number generator seed
total_mass = 20.0        # total mass of particles

n_particles = 500        # Number of particles
t = 0                    # current time of the simulation
t_end = 20.0              # time at which simulation ends
dt = 0.001                # timestep
softening = 0.1          # softening length
G = 1.0                  # Newton's Gravitational Constant

plot_real_time = True    # switch on for plotting as the simulation goes along
plot_energues = True
save_fig = False
figsize = 10

### The simulation

In [ ]:
# Generate Initial Conditions
np.random.seed(seed)            

mass = total_mass * np.ones((n_particles, 1)) / n_particles 

# randomly selected positions and velocities
pos = np.random.randn(n_particles, 3)   
vel = np.random.randn(n_particles, 3)

# Convert to Center-of-Mass frame
vel -= np.mean(mass * vel, 0) / np.mean(mass)

# calculate initial gravitational accelerations and energies 
acc = get_acceleration(pos, mass, G, softening)
KE, PE  = get_energy(pos, vel, mass, G)

# number of timesteps
n_timesteps = int(np.ceil(t_end / dt))

# save energies, particle orbits for plotting trails
pos_save = np.zeros((n_particles, 3, n_timesteps + 1))
pos_save[:, :, 0] = pos
KE_save = np.zeros(n_timesteps + 1)
KE_save[0] = KE
PE_save = np.zeros(n_timesteps + 1)
PE_save[0] = PE
t_all = np.arange(n_timesteps + 1) * dt

# prep figure
fig = plt.figure(figsize=(figsize, figsize))
grid = plt.GridSpec(3, 1, wspace=0.0, hspace=0.3)
ax1 = plt.subplot(grid[0:2, 0])
ax2 = plt.subplot(grid[2, 0])

# Simulation Main Loop
for i in range(n_timesteps):
    vel += acc * (dt / 2.0)  # (1/2) kick
    pos += vel * dt  # drift

    # update accelerations
    acc = get_acceleration(pos, mass, G, softening)

    vel += acc * (dt / 2.0)  # (1/2) kick

    t += dt  # update time

    # get energy of system
    KE, PE = get_energy(pos, vel, mass, G)

    # save energies, positions for plotting trail
    pos_save[:, :, i + 1] = pos
    KE_save[i + 1] = KE
    PE_save[i + 1] = PE

    # plot in real time
    if plot_real_time or (i == (n_timesteps - 1)):
        plt.sca(ax1)
        plt.cla()
        xx = pos_save[:, 0, max(i - 50, 0):(i + 1)]
        yy = pos_save[:, 1, max(i - 50, 0):(i + 1)]
        # plt.scatter(xx, yy, s=1, color=[.7, .7, 1])
        plt.scatter(pos[:, 0], pos[:, 1], s=2, color='black')
        # ax1.set_facecolor('black')
        ax1.set(xlim=(-2, 2), ylim=(-2, 2))
        ax1.set_aspect('equal', 'box')
        ax1.set_xticks([-2, -1, 0, 1, 2])
        ax1.set_yticks([-2, -1, 0, 1, 2])

        plt.sca(ax2)
        plt.cla()
        plt.scatter(t_all, KE_save, color='red', s=1, label='KE' if i == n_timesteps-1 else "")
        plt.scatter(t_all, PE_save, color='blue', s=1, label='PE' if i == n_timesteps-1 else "")
        plt.scatter(t_all, KE_save + PE_save, color='black', s=1, label='Etot' if i == n_timesteps-1 else "")
        ax2.set(xlim=(0, t_end), ylim=(-300, 300))
        ax2.set_aspect(0.007)

        # plt.pause(0.001)
        fig = plt.gcf()
        display(fig)
        clear_output(wait=True)

# add labels/legend
plt.sca(ax2)
plt.xlabel('time')
plt.ylabel('energy')
ax2.legend(loc='upper right')

# Save figure
if save_fig:
    plt.savefig('nbody.png',dpi=240)
    print(f'saved fig as nbody.png')
plt.show()